In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/eeg-misc-52-125hz/y_test.csv
/kaggle/input/eeg-misc-52-125hz/y_train.csv
/kaggle/input/eeg-misc-52-125hz/data.h5
/kaggle/input/eeg-misc-52-250hz/y_test.csv
/kaggle/input/eeg-misc-52-250hz/y_train.csv
/kaggle/input/eeg-misc-52-250hz/data_250hz.h5


In [2]:
import h5py
import wandb
wandb.login(key='0a4257fb4fa0625c3750f4f755c7ec9bdfd0c3c8')
y_train = pd.read_csv('/kaggle/input/eeg-misc-52-125hz/y_train.csv')
y_test = pd.read_csv('/kaggle/input/eeg-misc-52-125hz/y_test.csv')

train_data = []
test_data = []
train_label = []
test_label = []
with h5py.File('/kaggle/input/eeg-misc-52-125hz/data.h5', 'r') as f:
    min_len = 67750
    for (patient_id, age) in y_train.values:
        data = []
        for i in range(52):
            parcel = f[patient_id + '/parcel_' + str(i)][:]
            parcel = np.squeeze(parcel)
            data.append(parcel)
        data=np.array(data)
        if len(data[0]) == min_len:
            train_data.append(data)
            train_label.append(age)
        else:
            train_data.append(data[:, :min_len])
            train_data.append(data[:, -min_len:])
            train_label.append(age)
            train_label.append(age)
            
    for (patient_id, age) in y_test.values:
        data = []
        for i in range(52):
            parcel = f[patient_id + '/parcel_' + str(i)][:]
            parcel = np.squeeze(parcel)
            data.append(parcel)
        data=np.array(data)
        if len(data[0]) == min_len:
            test_data.append(data)
            test_label.append(age)
        else:
            test_data.append(data[:, :min_len])
            test_data.append(data[:, -min_len:])
            test_label.append(age)
            test_label.append(age)


wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: 22022500 (22022500-university-of-engineering-and-technology). Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [3]:
train_data = np.array(train_data)
test_data = np.array(test_data)
train_label = np.array(train_label)
test_label = np.array(test_label)
print(train_data.shape, test_data.shape)

(239, 52, 67750) (80, 52, 67750)


In [4]:
segment_length = 250
num_segments = train_data.shape[2] // segment_length

train_data_reshaped = train_data.reshape(-1, train_data.shape[1], segment_length) 
test_data_reshaped = test_data.reshape(-1, test_data.shape[1], segment_length)
train_label_reshaped = np.repeat(train_label, num_segments)  
test_label_reshaped = np.repeat(test_label, num_segments)   
print(train_data_reshaped.shape)

(64769, 52, 250)


In [5]:
labels = pd.read_csv('/kaggle/input/eeg-misc-52-125hz/y_train.csv').values.squeeze()

age_labels = labels[:, 1].astype(int)

num_classes = len(np.unique(age_labels))
print(f"Number of classes: {num_classes}")


Number of classes: 62


In [6]:
import torch
from torch.utils.data import Dataset, DataLoader
import pytorch_lightning as pl
import numpy as np

class EEGDatasetRegression(Dataset):
    def __init__(self, data, labels):
        self.data = data
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        x = self.data[idx]
        y = self.labels[idx]
        x = torch.tensor(x, dtype=torch.float32).unsqueeze(0)
        y = torch.tensor(y, dtype=torch.float32)
        return x, y

class EEGDataModule(pl.LightningDataModule):
    def __init__(self, train_data, train_labels, test_data, test_labels, batch_size=8):
        super().__init__()
        self.train_data = train_data
        self.train_labels = train_labels
        self.test_data = test_data
        self.test_labels = test_labels
        self.batch_size = batch_size

    def setup(self, stage=None):
        # Prepare datasets
        self.train_dataset = EEGDatasetRegression(self.train_data, self.train_labels)
        self.test_dataset = EEGDatasetRegression(self.test_data, self.test_labels)

    def train_dataloader(self):
        return DataLoader(self.train_dataset, batch_size=self.batch_size, shuffle=True)

    def val_dataloader(self):
        return DataLoader(self.test_dataset, batch_size=self.batch_size)

    def test_dataloader(self):
        return DataLoader(self.test_dataset, batch_size=self.batch_size)
batch_size = 8
eeg_dm = EEGDataModule(train_data, train_label, test_data, test_label, batch_size=batch_size)

In [7]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import pytorch_lightning as pl
from torchmetrics import MeanSquaredError, MeanAbsoluteError


class SelfAttentionBlock(nn.Module):

    def __init__(self, embed_dim, num_heads=4, dropout=0.1):
        super().__init__()
        self.mha = nn.MultiheadAttention(embed_dim, num_heads, dropout=dropout, batch_first=True)
        self.ln1 = nn.LayerNorm(embed_dim)
        
        self.ff = nn.Sequential(
            nn.Linear(embed_dim, embed_dim * 4),
            nn.ReLU(),
            nn.Linear(embed_dim * 4, embed_dim),
        )
        self.ln2 = nn.LayerNorm(embed_dim)
    
    def forward(self, x):

        x2, _ = self.mha(x, x, x)  # Q=K=V=x
        x = x + x2
        x = self.ln1(x)
        
        x2 = self.ff(x)
        x = x + x2
        x = self.ln2(x)
        
        return x


class CrossAttentionBlock(nn.Module):

    def __init__(self, embed_dim, num_heads=4, dropout=0.1):
        super().__init__()
        self.mha = nn.MultiheadAttention(embed_dim, num_heads, dropout=dropout, batch_first=True)
        self.ln1 = nn.LayerNorm(embed_dim)
        
        self.ff = nn.Sequential(
            nn.Linear(embed_dim, embed_dim * 4),
            nn.ReLU(),
            nn.Linear(embed_dim * 4, embed_dim),
        )
        self.ln2 = nn.LayerNorm(embed_dim)
    
    def forward(self, x):

        x2, _ = self.mha(x, x, x)  
        x = x + x2
        x = self.ln1(x)
        
        x2 = self.ff(x)
        x = x + x2
        x = self.ln2(x)
        
        return x

class EEGNetAgeRegressor(pl.LightningModule):
    def __init__(self,
                 Chans=22,          
                 Samples=256,        
                 F1=16,
                 D=2,
                 F2=8,
                 kernelLength=64,
                 dropoutRate=0.5,
                 learning_rate=1e-3,
                 weight_decay=1e-4,
                 num_heads_time=4,   
                 num_heads_chan=4):  
        super().__init__()
        
        self.save_hyperparameters()
        

        self.conv1 = nn.Conv2d(
            in_channels=1, 
            out_channels=F1,
            kernel_size=(1, kernelLength),
            padding=(0, kernelLength // 2),
            bias=False
        )
        self.bn1 = nn.BatchNorm2d(F1)
        
        self.depthwise_conv = nn.Conv2d(
            in_channels=F1,
            out_channels=F1*D,
            kernel_size=(Chans, 1),
            groups=F1,
            bias=False
        )
        self.bn2 = nn.BatchNorm2d(F1*D)
        
        self.elu = nn.ELU()
        
        self.pool1 = nn.AvgPool2d(kernel_size=(1, 4))
        self.dropout1 = nn.Dropout(dropoutRate)
        
        self.sep_conv1 = nn.Conv2d(
            in_channels=F1*D,
            out_channels=F1*D,
            kernel_size=(1, 16),
            padding=(0, 8),
            groups=F1*D,
            bias=False
        )
        self.bn3 = nn.BatchNorm2d(F1*D)
        
        self.sep_conv2 = nn.Conv2d(
            in_channels=F1*D,
            out_channels=F2,
            kernel_size=(1, 1),
            bias=False
        )
        self.bn4 = nn.BatchNorm2d(F2)
        
        self.pool2 = nn.AvgPool2d(kernel_size=(1, 8))
        self.dropout2 = nn.Dropout(dropoutRate)
        
 
        self.self_att_block = SelfAttentionBlock(embed_dim=F2, 
                                                 num_heads=num_heads_time, 
                                                 dropout=dropoutRate)
        
        self.cross_att_block = CrossAttentionBlock(embed_dim=F2, 
                                                   num_heads=num_heads_chan, 
                                                   dropout=dropoutRate)
        

        self.gap = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(F2, 1)
        
        self.train_mse = MeanSquaredError()
        self.val_mse = MeanSquaredError()
        self.test_mse = MeanSquaredError()
        self.test_mae = MeanAbsoluteError()

    def forward(self, x):
        """
        x: [B, 1, Chans, Samples]
        """
        # Khối EEGNet gốc
        x = self.conv1(x)          # -> [B, F1, Chans, Samples]
        x = self.bn1(x)
        
        x = self.depthwise_conv(x) # -> [B, F1*D, 1, Samples]
        x = self.bn2(x)
        x = self.elu(x)
        x = self.pool1(x)         # -> [B, F1*D, 1, Samples//4]
        x = self.dropout1(x)
        
        x = self.sep_conv1(x)     # -> [B, F1*D, 1, Samples//4]
        x = self.bn3(x)
        x = self.elu(x)
        
        x = self.sep_conv2(x)     # -> [B, F2, 1, Samples//4]
        x = self.bn4(x)
        x = self.elu(x)
        
        x = self.pool2(x)         # -> [B, F2, 1, newTime]
        x = self.dropout2(x)
        

        
        B, F2, dummy_chan, T = x.shape  
        
        x = x.view(B, F2, T)         
        x = x.permute(0, 2, 1)        
        
        x = self.self_att_block(x)    


        x = self.cross_att_block(x)    
        
        x = x.mean(dim=1)
        
        out = self.fc(x)  # [B, 1]
        return out.squeeze(-1) 
    
    def configure_optimizers(self):
        optimizer = torch.optim.Adam(
            self.parameters(),
            lr=self.hparams.learning_rate,
            weight_decay=self.hparams.weight_decay
        )
        return optimizer
    
    def training_step(self, batch, batch_idx):
        x, y = batch  
        preds = self.forward(x)
        loss = F.mse_loss(preds, y)
        self.log("train_loss", loss, on_epoch=True, prog_bar=True)
        
        mse_val = self.train_mse(preds, y)
        self.log("train_mse", mse_val, on_epoch=True, prog_bar=True)
        
        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        preds = self.forward(x)
        loss = F.mse_loss(preds, y)
        self.log("val_loss", loss, on_epoch=True, prog_bar=True)
        
        mse_val = self.val_mse(preds, y)
        self.log("val_mse", mse_val, on_epoch=True, prog_bar=True)
        
        return loss

    def test_step(self, batch, batch_idx):
        x, y = batch
        preds = self.forward(x)
        loss = F.mse_loss(preds, y)
        self.log("test_loss", loss, on_epoch=True, prog_bar=True)
        
        mse_val = self.test_mse(preds, y)
        self.log("test_mse", mse_val, on_epoch=True, prog_bar=True)

        mae_val = self.test_mae(preds, y)
        self.log("test_mae", mae_val, on_epoch=True, prog_bar=True)
        
        return loss


In [8]:
from pytorch_lightning.loggers import WandbLogger

wandb_logger = WandbLogger(project="EEG Competition", log_model=True)
checkpoint_callback = pl.callbacks.ModelCheckpoint(
    monitor="val_loss",
    mode="min",
    save_top_k=1,
    filename="best-model-{epoch}-{val_loss:.2f}"
)
model = EEGNetAgeRegressor(
    Chans=52,
    Samples=min_len,
    F1=8, D=2, F2=16,
    kernelLength=64,
    dropoutRate=0.5,
    learning_rate=1e-3,
    weight_decay=1e-4,
    num_heads_time=4,
    num_heads_chan=4
)

trainer = pl.Trainer(
    max_epochs=60, 
    accelerator="auto",  
    devices=2,
     logger=wandb_logger,
     callbacks=[checkpoint_callback]
)

trainer.fit(model, datamodule=eeg_dm)

trainer.test(model, datamodule=eeg_dm)


wandb: Tracking run with wandb version 0.19.1
wandb: Run data is saved locally in ./wandb/run-20241222_095340-9oty2e97
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run youthful-shape-32
wandb: ⭐️ View project at https://wandb.ai/22022500-university-of-engineering-and-technology/EEG%20competition
wandb: 🚀 View run at https://wandb.ai/22022500-university-of-engineering-and-technology/EEG%20competition/runs/9oty2e97


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/pytorch_lightning/trainer/connectors/logger_connector/result.py:431: It is recommended to use `self.log('val_loss', ..., sync_dist=True)` when logging on epoch level in distributed setting to accumulate the metric across devices.
/usr/local/lib/python3.10/dist-packages/pytorch_lightning/trainer/connectors/logger_connector/result.py:431: It is recommended to use `self.log('val_mse', ..., sync_dist=True)` when logging on epoch level in distributed setting to accumulate the metric across devices.
/usr/local/lib/python3.10/dist-packages/pytorch_lightning/loops/fit_loop.py:298: The number of training batches (15) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.


Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

wandb: Tracking run with wandb version 0.19.1
wandb: Run data is saved locally in ./wandb/run-20241222_100402-dd4mvlo1
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run balmy-snow-37
wandb: ⭐️ View project at https://wandb.ai/22022500-university-of-engineering-and-technology/EEG%20competition
wandb: 🚀 View run at https://wandb.ai/22022500-university-of-engineering-and-technology/EEG%20competition/runs/dd4mvlo1
/usr/local/lib/python3.10/dist-packages/pytorch_lightning/trainer/connectors/data_connector.py:215: Using `DistributedSampler` with the dataloaders. During `trainer.test()`, it is recommended to use `Trainer(devices=1, num_nodes=1)` to ensure each sample/batch gets evaluated exactly once. Otherwise, multi-device settings use `DistributedSampler` that replicates some samples to make sure all devices have same batch size in case of uneven inputs.


Testing: |          | 0/? [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/pytorch_lightning/trainer/connectors/logger_connector/result.py:431: It is recommended to use `self.log('test_loss', ..., sync_dist=True)` when logging on epoch level in distributed setting to accumulate the metric across devices.
/usr/local/lib/python3.10/dist-packages/pytorch_lightning/trainer/connectors/logger_connector/result.py:431: It is recommended to use `self.log('test_mse', ..., sync_dist=True)` when logging on epoch level in distributed setting to accumulate the metric across devices.
/usr/local/lib/python3.10/dist-packages/pytorch_lightning/trainer/connectors/logger_connector/result.py:431: It is recommended to use `self.log('test_mae', ..., sync_dist=True)` when logging on epoch level in distributed setting to accumulate the metric across devices.


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_loss         │     502.3091735839844     │
│         test_mae          │     19.04226303100586     │
│         test_mse          │     502.3091735839844     │
└───────────────────────────┴───────────────────────────┘

[{'test_loss': 502.3091735839844,
  'test_mse': 502.3091735839844,
  'test_mae': 19.04226303100586}]

In [9]:
import optuna
import pytorch_lightning as pl
from pytorch_lightning.callbacks import EarlyStopping

def objective(trial):
    F1 = trial.suggest_int("F1", 4, 32, step=4)
    D = trial.suggest_int("D", 1, 4, step=1)               
    F2 = trial.suggest_int("F2", 8, 64, step=8)           
    kernelLength = trial.suggest_int("kernelLength", 16, 128, step=16)
    
    dropoutRate = trial.suggest_float("dropoutRate", 0.1, 0.7, step=0.1)
    learning_rate = trial.suggest_float("learning_rate", 1e-4, 1e-2, log=True)
    weight_decay = trial.suggest_float("weight_decay", 1e-6, 1e-3, log=True)
    
    model = EEGNetAgeRegressor(
        Chans=52,
        Samples=min_len,         
        F1=F1,
        D=D,
        F2=F2,
        kernelLength=kernelLength,
        dropoutRate=dropoutRate,
        learning_rate=learning_rate,
        weight_decay=weight_decay
    )
    
    trainer = pl.Trainer(
        max_epochs=30,
        accelerator="auto",    
        devices=2,
        callbacks=[
            EarlyStopping(monitor="val_loss", mode="min", patience=5)
        ],
        enable_progress_bar=False  
    )
    
    trainer.fit(model, datamodule=eeg_dm)  
    
    val_loss = trainer.callback_metrics.get("val_loss")
    return val_loss.item()


In [10]:
study = optuna.create_study(direction="minimize")  # minimize val_loss
study.optimize(objective, n_trials=30)             # thử 30 trials
print("Number of finished trials:", len(study.trials))
print("Best trial:")
best_trial = study.best_trial
print("  Value: ", best_trial.value)

print("  Params: ")
for key, value in best_trial.params.items():
    print(f"    {key}: {value}")


[I 2024-12-22 10:04:05,333] A new study created in memory with name: no-name-8e57d462-6d77-4aa2-88b6-65ba4f68d434
/usr/local/lib/python3.10/dist-packages/pytorch_lightning/trainer/connectors/logger_connector/result.py:431: It is recommended to use `self.log('val_loss', ..., sync_dist=True)` when logging on epoch level in distributed setting to accumulate the metric across devices.
/usr/local/lib/python3.10/dist-packages/pytorch_lightning/trainer/connectors/logger_connector/result.py:431: It is recommended to use `self.log('val_mse', ..., sync_dist=True)` when logging on epoch level in distributed setting to accumulate the metric across devices.
/usr/local/lib/python3.10/dist-packages/pytorch_lightning/loops/fit_loop.py:298: The number of training batches (15) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.
/usr/local/lib/python3.10/dist-packages/pytorch_lightning/trainer/connecto

Number of finished trials: 30
Best trial:
  Value:  447.27215576171875
  Params: 
    F1: 8
    D: 2
    F2: 24
    kernelLength: 64
    dropoutRate: 0.5
    learning_rate: 0.0042234270303314
    weight_decay: 5.032469622857439e-05


In [11]:
optimal_params = study.best_trial.params

model = EEGNetAgeRegressor(
    Chans=52,
    Samples=256,
    F1=optimal_params["F1"],
    D=optimal_params["D"],
    F2=optimal_params["F2"],
    kernelLength=optimal_params["kernelLength"],
    dropoutRate=optimal_params["dropoutRate"],
    learning_rate=optimal_params["learning_rate"],
    weight_decay=optimal_params["weight_decay"]
)

trainer = pl.Trainer(max_epochs=40, accelerator="auto", devices=2)
trainer.fit(model, datamodule=eeg_dm)
trainer.test(model, datamodule=eeg_dm)


/usr/local/lib/python3.10/dist-packages/pytorch_lightning/trainer/connectors/logger_connector/result.py:431: It is recommended to use `self.log('val_loss', ..., sync_dist=True)` when logging on epoch level in distributed setting to accumulate the metric across devices.
/usr/local/lib/python3.10/dist-packages/pytorch_lightning/trainer/connectors/logger_connector/result.py:431: It is recommended to use `self.log('val_mse', ..., sync_dist=True)` when logging on epoch level in distributed setting to accumulate the metric across devices.
/usr/local/lib/python3.10/dist-packages/pytorch_lightning/loops/fit_loop.py:298: The number of training batches (15) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.


Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/pytorch_lightning/trainer/connectors/logger_connector/result.py:431: It is recommended to use `self.log('train_loss', ..., sync_dist=True)` when logging on epoch level in distributed setting to accumulate the metric across devices.
/usr/local/lib/python3.10/dist-packages/pytorch_lightning/trainer/connectors/logger_connector/result.py:431: It is recommended to use `self.log('train_mse', ..., sync_dist=True)` when logging on epoch level in distributed setting to accumulate the metric across devices.


Validation: |          | 0/? [00:00<?, ?it/s]

...

/usr/local/lib/python3.10/dist-packages/pytorch_lightning/trainer/connectors/data_connector.py:215: Using `DistributedSampler` with the dataloaders. During `trainer.test()`, it is recommended to use `Trainer(devices=1, num_nodes=1)` to ensure each sample/batch gets evaluated exactly once. Otherwise, multi-device settings use `DistributedSampler` that replicates some samples to make sure all devices have same batch size in case of uneven inputs.


Testing: |          | 0/? [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/pytorch_lightning/trainer/connectors/logger_connector/result.py:431: It is recommended to use `self.log('test_loss', ..., sync_dist=True)` when logging on epoch level in distributed setting to accumulate the metric across devices.
/usr/local/lib/python3.10/dist-packages/pytorch_lightning/trainer/connectors/logger_connector/result.py:431: It is recommended to use `self.log('test_mse', ..., sync_dist=True)` when logging on epoch level in distributed setting to accumulate the metric across devices.
/usr/local/lib/python3.10/dist-packages/pytorch_lightning/trainer/connectors/logger_connector/result.py:431: It is recommended to use `self.log('test_mae', ..., sync_dist=True)` when logging on epoch level in distributed setting to accumulate the metric across devices.


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_loss         │    447.89495849609375     │
│         test_mae          │    18.624998092651367     │
│         test_mse          │    447.89495849609375     │
└───────────────────────────┴───────────────────────────┘

[{'test_loss': 447.89495849609375,
  'test_mse': 447.89495849609375,
  'test_mae': 18.624998092651367}]